---

# **Notebook : Analyse et Nettoyage de Données Open Food Facts avec Chunks**

---


# **Phase 1**

---

### **1. Importation des bibliothèques**

La première section est dédiée à l'importation des bibliothèques nécessaires pour manipuler les fichiers et les données.



# 1. Importation des bibliothèques


In [7]:

import os
import pandas as pd

# Bibliothèques nécessaires :
# - os : pour gérer les fichiers et répertoires.
# - pandas : pour lire et manipuler les données CSV.

---

### **2. Définir le chemin du fichier CSV et la taille des chunks**

Dans cette étape, nous définissons le chemin vers le fichier CSV et la taille des chunks que nous voulons créer.

# 2. Définir le chemin du fichier CSV et la taille des chunks

# Chemin du fichier CSV à découper

In [23]:
import os
path = os.getcwd()
print(path)
os.chdir('note-books/phase_1_data')

/Users/mwrevent/Dropbox/Mon Mac (Romualdraphique)/Desktop/code/Simplon/brief/Projet-OpenFood/documents_other


In [8]:
csv_file_path = 'en.openfoodfacts.org.products.csv'  # Assurez-vous que ce fichier est bien dans votre répertoire


# Taille des chunks (100 000 lignes par chunk)

In [9]:
chunk_size = 100000

# Chemin du dossier de sortie où les chunks seront stockés

In [10]:
output_directory = '02_chunk-CSV'

### **3. Lecture du CSV et division en chunks**

Ici, nous implémentons la fonction qui va lire le fichier CSV et le diviser en plusieurs morceaux de 100 000 lignes, puis sauvegarder chaque chunk dans le dossier `02_chunk-CSV`.


# Fonction pour lire et diviser le fichier CSV en chunks

In [11]:
def diviser_fichier_en_chunks(file_path, chunk_size, output_directory):
    # Lire le fichier CSV en morceaux (chunks)
    chunk_iter = pd.read_csv(file_path, sep='\t', chunksize=chunk_size, low_memory=False)
    
    for i, chunk in enumerate(chunk_iter):
        # Sauvegarder chaque chunk dans un fichier CSV distinct
        chunk_file_name = os.path.join(output_directory, f'chunk_{i+1}.csv')
        chunk.to_csv(chunk_file_name, index=False)
        print(f'Chunk {i+1} sauvegardé dans {chunk_file_name}.')


# Exécuter le découpage en chunks

In [12]:
diviser_fichier_en_chunks(csv_file_path, chunk_size, output_directory)

Chunk 1 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_1.csv.
Chunk 2 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_2.csv.
Chunk 3 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_3.csv.
Chunk 4 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_4.csv.
Chunk 5 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_5.csv.
Chunk 6 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_6.csv.
Chunk 7 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_7.csv.
Chunk 8 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_8.csv.
Chunk 9 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_9.csv.
Chunk 10 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_10.csv.
Chunk 11 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_11.csv.
Chunk 12 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_12.csv.
Chunk 13 sauvegardé dans note-books/phase_1_data/02_chunk-CSV/chunk_13.csv.
Chunk 14 sauvegardé dans note-

ParserError: Error tokenizing data. C error: Expected 206 fields in line 1600375, saw 234


### **4. Conclusion**

À ce stade, votre fichier CSV sera divisé en chunks de 100 000 lignes, et chaque chunk sera sauvegardé dans le dossier `02_chunk-CSV`. Vous pouvez vérifier le dossier pour voir les fichiers générés.


### Résumé des étapes

1. **Importation des bibliothèques** : Importer `os` et `pandas` pour gérer les fichiers et manipuler les données.
2. **Définir le chemin du fichier CSV et la taille des chunks** : Spécifier le fichier CSV à diviser et la taille des chunks (100 000 lignes).
3. **Lecture et division en chunks** : Diviser le fichier CSV en chunks et sauvegarder chaque chunk dans le dossier `02_chunk-CSV`.
